# JACTUS: Getting Started with Financial Contracts

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pedronahum/JACTUS/blob/main/examples/notebooks/00_getting_started_pam.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-JACTUS-blue?logo=github)](https://github.com/pedronahum/JACTUS)
[![PyPI](https://img.shields.io/pypi/v/jactus)](https://pypi.org/project/jactus/)
[![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](https://opensource.org/licenses/Apache-2.0)

**JACTUS** is a high-performance Python library implementing the [ACTUS](https://www.actusfrf.org/) (Algorithmic Contract Types Unified Standards) financial contract specification using [JAX](https://jax.readthedocs.io/).

In this notebook, you will:
1. Install JACTUS
2. Create a **Principal at Maturity (PAM)** contract — a simple interest-only loan
3. Simulate the contract and inspect the generated cash flow events
4. Understand the ACTUS event lifecycle

## 1. Install JACTUS

JACTUS is available on [PyPI](https://pypi.org/project/jactus/). The installation takes about a minute.

In [ ]:
!pip install -q jactus

In [ ]:
import jactus
print(f"JACTUS version: {jactus.__version__}")

## 2. What is a PAM Contract?

A **Principal at Maturity (PAM)** contract is the simplest ACTUS contract type. It represents an interest-only loan or bond where:

- The **full principal** is exchanged at the start (Initial Exchange Date)
- **Interest payments** are made periodically (e.g., monthly, quarterly, semi-annually)
- The **full principal** is returned at maturity along with the final interest payment
- No principal amortization occurs during the life of the contract

**Real-world examples:** Treasury bonds, corporate bonds, bullet loans, interest-only mortgages.

## 3. Create the Contract

We'll model a **$100,000 loan at 5% annual interest** with semi-annual interest payments and a 1-year maturity.

In [ ]:
from jactus.contracts import create_contract
from jactus.core import (
    ContractAttributes,
    ContractType,
    ContractRole,
    ActusDateTime,
)
from jactus.observers import ConstantRiskFactorObserver

### 3.1 Define Contract Attributes

Every ACTUS contract is defined by a set of **attributes** — parameters that describe the contract terms. Here are the key ones for a PAM:

| Attribute | Description | Our Value |
|-----------|-------------|-----------|
| `contract_type` | ACTUS contract type | PAM |
| `contract_role` | RPA (lender) or RPL (borrower) | RPA |
| `status_date` | Observation/valuation date | 2024-01-01 |
| `initial_exchange_date` | When principal is disbursed | 2024-01-15 |
| `maturity_date` | When principal is returned | 2025-01-15 |
| `notional_principal` | Loan amount | $100,000 |
| `nominal_interest_rate` | Annual interest rate | 5% |
| `interest_payment_cycle` | Payment frequency | 6M (semi-annual) |
| `day_count_convention` | How days are counted for interest | 30E/360 |

In [ ]:
attrs = ContractAttributes(
    contract_id="PAM-EXAMPLE-001",
    contract_type=ContractType.PAM,
    contract_role=ContractRole.RPA,     # We are the lender (receive principal back)
    status_date=ActusDateTime(2024, 1, 1),
    initial_exchange_date=ActusDateTime(2024, 1, 15),
    maturity_date=ActusDateTime(2025, 1, 15),
    notional_principal=100_000.0,
    nominal_interest_rate=0.05,          # 5% annual
    interest_payment_cycle="6M",         # Semi-annual interest payments
    day_count_convention="30E360",        # 30E/360 day count
)

print(f"Contract ID:    {attrs.contract_id}")
print(f"Contract Type:  {attrs.contract_type.value}")
print(f"Role:           {attrs.contract_role.value} (lender)")
print(f"Principal:      ${attrs.notional_principal:,.2f}")
print(f"Interest Rate:  {attrs.nominal_interest_rate:.1%}")
print(f"Maturity:       1 year")

### 3.2 Create a Risk Factor Observer

ACTUS contracts can depend on external **risk factors** (market rates, FX rates, etc.). For a fixed-rate PAM, we use a `ConstantRiskFactorObserver` — it simply returns a constant value whenever queried.

In [ ]:
rf_observer = ConstantRiskFactorObserver(constant_value=0.0)
print("Risk factor observer: ConstantRiskFactorObserver (constant_value=0.0)")

### 3.3 Create the Contract

The `create_contract` factory function takes the attributes and risk factor observer and returns the appropriate contract implementation.

In [ ]:
contract = create_contract(attrs, rf_observer)
print(f"Contract class: {type(contract).__name__}")

## 4. Simulate the Contract

Calling `contract.simulate()` runs the ACTUS simulation engine. It:
1. Initializes the contract state
2. Generates the event schedule (IED, IP, MD, etc.)
3. Processes each event chronologically, updating state and computing payoffs
4. Returns a `SimulationHistory` with all events and states

In [ ]:
result = contract.simulate()

print(f"Simulation complete!")
print(f"Total events generated: {len(result.events)}")
print(f"Final notional:         ${float(result.final_state.nt):,.2f}")

## 5. Inspect the Events

Each event in the simulation has:
- **`event_type`** — What happened (IED, IP, MD, etc.)
- **`event_time`** — When it happened
- **`payoff`** — The cash flow amount (positive = inflow, negative = outflow, from the lender's perspective)
- **`currency`** — Currency of the cash flow

### ACTUS Event Types

| Event | Name | Description |
|-------|------|-------------|
| **AD** | Analysis Date | Status observation (no cash flow) |
| **IED** | Initial Exchange | Principal disbursement |
| **IP** | Interest Payment | Periodic interest payment |
| **MD** | Maturity | Principal repayment + final interest |

In [ ]:
print("Cash Flow Schedule")
print("=" * 70)
print(f"{'Date':<14} {'Event':<8} {'Type':<28} {'Payoff':>14}")
print("-" * 70)

event_descriptions = {
    "AD":  "Analysis/Status Date",
    "IED": "Initial Exchange (Disbursement)",
    "IP":  "Interest Payment",
    "MD":  "Maturity (Principal + Interest)",
}

for event in result.events:
    t = event.event_time
    date_str = f"{t.year}-{t.month:02d}-{t.day:02d}"
    etype = event.event_type.name
    desc = event_descriptions.get(etype, etype)
    payoff = float(event.payoff)
    payoff_str = f"${payoff:>12,.2f}" if payoff != 0 else f"{'—':>13}"
    print(f"{date_str:<14} {etype:<8} {desc:<28} {payoff_str}")

print("=" * 70)

### Understanding the Cash Flows

From the **lender's perspective** (RPA role):

1. **IED** (Initial Exchange): The lender disburses $100,000 to the borrower → **negative** cash flow
2. **IP** (Interest Payment): The borrower pays interest → **positive** cash flow
3. **MD** (Maturity): The borrower returns the principal plus final interest → **positive** cash flow

The interest for a 6-month period with 5% annual rate on $100,000 using 30E/360:

$$\text{Interest} = \$100{,}000 \times 0.05 \times \frac{180}{360} = \$2{,}500$$

In [ ]:
# Summarize the cash flows
total_inflows = sum(float(e.payoff) for e in result.events if float(e.payoff) > 0)
total_outflows = sum(float(e.payoff) for e in result.events if float(e.payoff) < 0)
net = total_inflows + total_outflows

print("Cash Flow Summary (Lender's Perspective)")
print("=" * 45)
print(f"Total disbursed (outflow):  ${total_outflows:>12,.2f}")
print(f"Total received (inflow):    ${total_inflows:>12,.2f}")
print(f"                            {'-' * 12}")
print(f"Net profit:                 ${net:>12,.2f}")
print(f"\nReturn on investment:        {net / abs(total_outflows):.2%}")

## 6. Inspect Contract State

The ACTUS simulation maintains a **contract state** that evolves with each event. Key state variables:

| Variable | Description |
|----------|-------------|
| `nt` | Outstanding notional (principal) |
| `ipnr` | Nominal interest rate |
| `ipac` | Accrued interest |
| `sd` | Last status date |

In [ ]:
print("Contract State Evolution")
print("=" * 75)
print(f"{'Date':<14} {'Event':<8} {'Notional':>14} {'Rate':>8} {'Accrued Int':>14}")
print("-" * 75)

# Show initial state
s = result.initial_state
print(f"{'(initial)':<14} {'—':<8} ${float(s.nt):>12,.2f} {float(s.ipnr):>7.2%} ${float(s.ipac):>12,.2f}")

# Show state after each event
for event in result.events:
    t = event.event_time
    date_str = f"{t.year}-{t.month:02d}-{t.day:02d}"
    s = result.states[result.events.index(event)]
    print(f"{date_str:<14} {event.event_type.name:<8} ${float(s.nt):>12,.2f} {float(s.ipnr):>7.2%} ${float(s.ipac):>12,.2f}")

print("=" * 75)

## 7. Next Steps

Now that you've seen a basic PAM contract, explore more JACTUS capabilities:

| Contract Type | Description | Example |
|--------------|-------------|----------|
| **ANN** | Annuity (mortgage) | [`01_annuity_mortgage.ipynb`](https://colab.research.google.com/github/pedronahum/JACTUS/blob/main/examples/notebooks/01_annuity_mortgage.ipynb) |
| **OPTNS** | Options (calls/puts) | [`02_options_contracts.ipynb`](https://colab.research.google.com/github/pedronahum/JACTUS/blob/main/examples/notebooks/02_options_contracts.ipynb) |
| **CAPFL** | Interest rate caps | [`03_interest_rate_cap.ipynb`](https://colab.research.google.com/github/pedronahum/JACTUS/blob/main/examples/notebooks/03_interest_rate_cap.ipynb) |
| **STK/COM** | Stocks & commodities | [`04_stock_commodity.ipynb`](https://colab.research.google.com/github/pedronahum/JACTUS/blob/main/examples/notebooks/04_stock_commodity.ipynb) |

### Resources

- **Documentation**: [pedronahum.github.io/JACTUS](https://pedronahum.github.io/JACTUS/)
- **GitHub**: [github.com/pedronahum/JACTUS](https://github.com/pedronahum/JACTUS)
- **PyPI**: [pypi.org/project/jactus](https://pypi.org/project/jactus/)
- **ACTUS Standard**: [actusfrf.org](https://www.actusfrf.org/)